In [1]:
import numpy as np
import sys
from transformers_encoder import TransformersEncoder
from transformers import RobertaModel, RobertaTokenizer
from vectorspace import SensesVSM
import spacy
en_nlp = spacy.load('en_core_web_trf')  # required for lemmatization and POS-tagging 
import torch
from wn_utils import WN_Utils
wn_utils = WN_Utils()  # WordNet auxilliary methods (just for describing results)
basePath = '/data/LLMs/LMMS/'

/home3/ebrahim/miniconda3/envs/llama/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# NLM/LMMS paths and parameters
vecs_path = f'{basePath}data/vectors/lmms-sp-wsd.roberta-large.vectors.txt'
wsd_encoder_cfg = {
    'model_name_or_path': 'roberta-large',
    'min_seq_len': 0,
    'max_seq_len': 512,
    'layers': [-n for n in range(1, 12 + 1)],  # all layers, with reversed indices
    'layer_op': 'ws',
    'weights_path': f'{basePath}data/weights/lmms-sp-wsd.roberta-large.weights.txt',
    'subword_op': 'mean'
}

In [4]:
model_str = 'roberta-large'
tokenizer = RobertaTokenizer.from_pretrained(model_str)
model = RobertaModel.from_pretrained(model_str)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
print('Loading NLM and sense embeddings ...')  # (takes a while)
wsd_encoder = TransformersEncoder(wsd_encoder_cfg)
senses_vsm = SensesVSM(vecs_path, normalize=True)
print('Done')

Loading NLM and sense embeddings ...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Done


In [7]:
print("Memory size of numpy array in GB:",
      (senses_vsm.vectors.size * senses_vsm.vectors.itemsize)/1024**3)

Memory size of numpy array in GB: 0.7894477844238281


In [8]:
senses_vsm.vectors.shape

(206949, 1024)

In [9]:
embedding_vectors = np.random.randn(20, 1024).T
embedding_vectors.shape
sim_scores = senses_vsm.vectors@embedding_vectors

In [21]:
sim_scores[:, 1].max()

2.655784968568071

In [26]:
sorted_idxs_t5 = np.argsort(sim_scores, axis=0)[-5:, :]
print(sorted_idxs_t5[:, 1]
sim_scores[sorted_idxs_t5[-1, 1], 1]

1.1437644562727962

In [ ]:
senses_vsm.vectors[sorted_idxs_t5, :][-1].shape

In [18]:
sorted_idxs_t5[:, 0]

array([105473,  17723, 121972,  61128,  96400])